# 4.6 Lab: Logistic Regression, LDA, QDA, and KNN

## 4.6.1 The Stock Market Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import math
from patsy import dmatrices


import statsmodels.discrete.discrete_model as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model
from sklearn.metrics import confusion_matrix

In [ ]:
Smarket = pd.read_csv('data/Smarket.csv', header=0)

In [ ]:
Smarket.head()

In [ ]:
list(Smarket)

In [ ]:
Smarket.shape

In [ ]:
"""
For panda data frame, there is a method corr to compute pairwise correlation between numerical variables
"""
Smarket.corr()

In [ ]:
"""
As one would expect, the correlations between the lag variables and today’s returns are close to zero. 
"""
plt.plot(Smarket.ix[:, 6])
#plt.plot(Smarket[['Volume']])
plt.show()

## 4.6.2 Logistic Regression

## There is some known complications that in Sklearn about applying parameter regularization. This can be aviod to set the tuning parameter 'C' to a large number. Here to be consistent with R output, I decieded to use Statsmodels.

In [ ]:
y, X = dmatrices('Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume', Smarket, return_type = 'dataframe')
print y

In [ ]:
"""
Since we are more interested in stock marketing up, we take the second column of y as our response
"""
logit = sm.Logit(y.ix[:,1], X)
logit.fit().summary()

In [ ]:
logit.fit().params # To extract the parameters directly

In [ ]:
logit.fit().predict()[0:11] # To extract the probability of the market going up

### In order to make a prediction as to whether the market will go up or down on a particular day, we must convert these predicted probabilities into class labels, Up (1) or Down (0).

In [ ]:
predict_label = pd.DataFrame(np.zeros(shape=(1250,1)), columns = ['label'])

In [ ]:
predict_label.ix[logit.fit().predict()>0.5] = 1

### We can evalue the TRAINING result by constructing a confusion matrix 

In [ ]:
confusion_matrix(y.ix[:,1], predict_label.ix[:,0])

### The diagonal elements of the confusion matrix indicate correct predictions, while the off-diagonals represent incorrect predictions. In this case, logistic regression correctly predicted the movement of the market 52.2% of the time.

In [ ]:
np.mean(y.ix[:,1] == predict_label.ix[:,0]) # to get accuracy

### In order to better assess the accuracy of the logistic regression model in this setting, we can fit the model using part of the data, and then examine how well it predicts the held out data. This will yield a more realistic error rate, in the sense that in practice we will be interested in our model’s performance not on the data that we used to fit the model, but rather on days in the future for which the market’s movements are unknown.

In [ ]:
Smarket_2005 = Smarket.query('Year >= 2005')
Smarket_train = Smarket.query('Year < 2005')

### We will use the training dataset to build the logistic regression model 

In [ ]:
y_train, X_train = dmatrices('Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume', Smarket_train, return_type = 'dataframe')
y_test, X_test = dmatrices('Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume', Smarket_2005, return_type = 'dataframe')

In [ ]:
logit = sm.Logit(y_train.ix[:,1], X_train)
print logit.fit().summary()

In [ ]:
preds = logit.fit().predict(X_test)
predict_label = pd.DataFrame(np.zeros(shape=(X_test.shape[0],1)), columns = ['label'])
threshold = 0.5
mark = (preds > threshold).reset_index(drop=True)
predict_label.ix[mark] = 1
confusion_matrix(y_test.ix[:,1], predict_label.ix[:,0])

In [ ]:
np.mean(y_test.iloc[:,1].reset_index(drop=True)==predict_label.iloc[:,0].reset_index(drop=True)) # to get accuracy

### Notice that we have trained and tested our model on two completely separate data sets: training was performed using only the dates before 2005, and testing was performed using only the dates in 2005. Finally, we compute the predictions for 2005 and compare them to the actual movements of the market over that time period. The results are rather disappointing: the test error rate is 1 - 48% = 52 %, which is worse than random guessing! Of course this result is not all that surprising, given that one would not generally expect to be able to use previous days’ returns to predict future market performance.

### The retrain of the model with Lag1 and Lag2 will be similar to previous steps (I will omit those). Another way to deal with logistics regression is to change the threshold value from 0.5 to others. There is an example below with threshold 0.45. 

In [ ]:
preds = logit.fit().predict(X_test)
predict_label = pd.DataFrame(np.zeros(shape=(X_test.shape[0],1)), columns = ['label'])
threshold = 0.45
predict_label.ix[(preds > threshold).reset_index(drop=True)] = 1
confusion_matrix(y_test.ix[:,1], predict_label.ix[:,0])
np.mean(y_test.iloc[:,1].reset_index(drop=True)==predict_label.iloc[:,0].reset_index(drop=True)) # to get accuracy

In [ ]:
y_train, X_train = dmatrices('Direction~Lag1+Lag2', Smarket_train, return_type = 'dataframe')
y_test, X_test = dmatrices('Direction~Lag1+Lag2', Smarket_2005, return_type = 'dataframe')
logit = sm.Logit(y_train.ix[:,1], X_train)
preds = logit.fit().predict(X_test)
predict_label = pd.DataFrame(np.zeros(shape=(X_test.shape[0],1)), columns = ['label'])
threshold = 0.5
confusion_matrix(y_test.ix[:,1], predict_label.ix[:,0])
np.mean(y_test.iloc[:,1].reset_index(drop=True)==predict_label.iloc[:,0].reset_index(drop=True)) # to get accuracy

## 4.6.3 Linear Discriminant Analysis

### We will use sklearn's implementation of LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

### The training process 

In [ ]:
sklearn_lda = LDA(n_components=2) #creating a LDA object
lda = sklearn_lda.fit(X_train.ix[:,1:3], y_train.ix[:,1]) #learning the projection matrix
X_lda = lda.transform(X_train.ix[:,1:3]) #using the model to project X 
X_labels = lda.predict(X_train.ix[:,1:3]) #gives you the predicted label for each sample
X_prob = lda.predict_proba(X_train.ix[:,1:3]) #the probability of each sample to belong to each class

### Testing step 

In [ ]:
X_test_labels=lda.predict(X_test.ix[:,1:3])
X_test_prob = lda.predict_proba(X_test.ix[:,1:3]) 

### Get the accuracy of the test set 

In [ ]:
np.mean(y_test.ix[:,1]==X_test_labels) 

### Let's change the threshod a bit to see whether we can improve the accuracy. The 2nd column of X_test_prob is the probability belongs to UP group. The default value is 0.5, let us first check that. 

In [ ]:
threshold = 0.5 
np.mean(y_test.ix[:,1]==(X_test_prob[:,1]>=threshold))

In [ ]:
threshold = 0.48
np.mean(y_test.ix[:,1]==(X_test_prob[:,1]>=threshold))

## 4.6.4 Quadratic Discriminant Analysis

### It is a little bit of annoying that QDA and LDA have minor difference in their parameter set-up and function names. 

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

In [ ]:
sklearn_qda = QDA(priors=None,store_covariances=True) #creating a QDA object
qda = sklearn_qda.fit(X_train.ix[:,1:3], y_train.ix[:,1]) #learning the projection matrix
X_labels = qda.predict(X_train.ix[:,1:3]) #gives you the predicted label for each sample
X_prob = qda.predict_proba(X_train.ix[:,1:3]) #the probability of each sample to belong to each class

X_test_labels=qda.predict(X_test.ix[:,1:3])
X_test_prob = qda.predict_proba(X_test.ix[:,1:3]) 

print np.mean(y_test.ix[:,1]==X_test_labels) 

### Again, use dir() to explore all the information stored in lda and qda.

In [ ]:
dir(qda)

In [ ]:
print qda.means_
print qda.covariances_

### 4.6.5 K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [ ]:
neigh = KNN(n_neighbors= 3) # use n_neighbors to change the # of tune the performance of KNN
KNN_fit = neigh.fit(X_train.ix[:,1:3], y_train.ix[:,1]) #learning the projection matrix
X_test_labels=KNN_fit.predict(X_test.ix[:,1:3])
X_test_prob = KNN_fit.predict_proba(X_test.ix[:,1:3]) 

print np.mean(y_test.ix[:,1]==X_test_labels) 

dir(neigh) # use dir command to check what KNN offers

## 4.6.6 An Application to Caravan Insurance Data

In [ ]:
Caravan = pd.read_csv('data/Caravan.csv', header=0)

In [ ]:
Caravan.shape

In [ ]:
Caravan.head()

In [ ]:
Caravan.describe()

### Scale of the variables matters in KNN ! The core question in KNN is how to define proper distance metric. 

### Because the KNN classifier predicts the class of a given test observation by identifying the observations that are nearest to it, the scale of the variables matters. Any variables that are on a large scale will have a much larger effect on the distance between the observations, and hence on the KNN classifier, than variables that are on a small scale. For instance, imagine a data set that contains two variables, salary and age (measured in dollars and years, respectively). As far as KNN is concerned, a difference of 1,000 in salary is enormous compared to a difference of 50 years in age. Consequently, salary will drive the KNN classification results, and age will have almost no effect. This is contrary to our intuition that a salary difference of 1, 000 is quite small compared to an age difference of 50 years. Furthermore, the importance of scale to the KNN classifier leads to another issue: if we measured salary in Japanese yen, or if we measured age in minutes, then we’d get quite different classification results from what we get if these two variables are measured in dollars and years. 

### A good way (?) to handle this problem is to standardize the data so that all standardize variables are given a mean of zero and a standard deviation of one. Then all variables will be on a comparable scale. The scale() function does just scale() this. In standardizing the data, we exclude column 86, because that is the qualitative Purchase variable.

In [ ]:
predict_label = pd.DataFrame(np.zeros(shape=(Caravan.shape[0],1)), columns = ['label'])
predict_label.ix[Caravan['Purchase'] == 'Yes'] = 1
Caravan_drop = Caravan.drop(labels='Purchase', axis=1)

### I took a slightly different approach from the book. The training and testing data were splited by index. The normalization was done on the train set. Afterwards, the same normalization was applied to validate test.  The code might seem wordy, but it helps clear the logical flow. 

In [ ]:
from sklearn import preprocessing
train_size = 1000
train_index = xrange(0, train_size)
X_validate = Caravan_drop.ix[train_index, ]
Y_validate = predict_label.ix[train_index, ]
X_train = Caravan_drop.ix[train_size:, ]
Y_train = predict_label.ix[train_size:, ]


X_train_scaled = preprocessing.scale(X_train)
scaler = preprocessing.StandardScaler().fit(X_train)
X_validate_scaled = scaler.transform(X_validate)   

### Train with 1 neighbor 

In [ ]:
neigh = KNN(n_neighbors= 1) # use n_neighbors to change the # of tune the performance of KNN
KNN_fit = neigh.fit(X_train_scaled, Y_train.ix[:,0]) #learning the projection matrix
X_validate_labels=KNN_fit.predict(X_validate_scaled)
X_validate_prob = KNN_fit.predict_proba(X_validate_scaled) 

print np.mean(Y_validate.ix[:,0]==X_validate_labels) 
print confusion_matrix(Y_validate.ix[:,0], X_validate_labels)

### The rest of this exercise considers all the trade-off between False postive and False negative.  The concept of accuracy is NOT always the golden metric for classification problems. 
### Precision and recall, sensitivity and specificity, F1 score... are all reasonable metrics to consider. 
### We will discuss more on the concept of trainning, validation and test. 